## Aprendizado Profundo

### O Tensor

In [1]:
Tensor = list

from typing import List

def shape(tensor: Tensor) -> List[int]:
    sizes: List[int] = []
    while isinstance(tensor, list):
        sizes.append(len(tensor))
        tensor = tensor[0]

    return sizes

assert shape([1, 2, 3]) == [3]
assert shape([[1, 2], [3, 4], [5, 6]]) == [3, 2]

In [2]:
def is_1d(tensor: Tensor) -> bool:
    """
    Se o tensor [0] é uma lista, é um tensor de ordem superior.
    Se não, o tensor é unidimensional (ou seja, um vetor).
    """
    return not isinstance(tensor[0], list)

assert is_1d([1, 2, 3])
assert not is_1d([[1, 2], [3, 4]])

In [4]:
def tensor_sum(tensor: Tensor) -> float:
    """Soma todos os valores do tensor"""
    if is_1d(tensor):
        return sum(tensor)                  # apenas uma lista de floats, use a soma Python
    else:
        return sum(tensor_sum(tensor_i)     # Chame tensor_sum em cada linha
                   for tensor_i in tensor)  # e some esses resultados

assert tensor_sum([1, 2, 3]) == 6
assert tensor_sum([[1, 2], [3, 4]]) == 10

In [5]:
from typing import Callable

def tensor_apply(f: Callable[[float], float], tensor: Tensor) -> Tensor:
    """Aplica f elementwise"""
    if is_1d(tensor):
        return [f(x) for x in tensor]
    else:
        return [tensor_apply(f, tensor_i) for tensor_i in tensor]

assert tensor_apply(lambda x: x + 1, [1, 2, 3]) == [2, 3, 4]
assert tensor_apply(lambda x: 2 * x, [[1, 2], [3, 4]]) == [[2, 4], [6, 8]]

In [6]:
def zeros_like(tensor: Tensor) -> Tensor:
    return tensor_apply(lambda _: 0.0, tensor)

assert zeros_like([1, 2, 3]) == [0, 0, 0]
assert zeros_like([[1, 2], [3, 4]]) == [[0, 0], [0, 0]]

In [7]:
def tensor_combine(f: Callable[[float, float], float],
                   t1: Tensor,
                   t2: Tensor) -> Tensor:
    """Aplica f aos elementos correspondentes de t1 e t2"""
    if is_1d(t1):
        return [f(x, y) for x, y in zip(t1, t2)]
    else:
        return [tensor_combine(f, t1_i, t2_i)
                for t1_i, t2_i in zip(t1, t2)]

import operator
assert tensor_combine(operator.add, [1, 2, 3], [4, 5, 6]) == [5, 7, 9]
assert tensor_combine(operator.mul, [1, 2, 3], [4, 5, 6]) == [4, 10, 18]

### Abstração de Camadas

In [58]:
from typing import Iterable, Tuple

class Layer:
    """
    Nossas redes neurais serão compostas por Layers que sabem
    computar as entradas "pra frente" e propagar gradientes "para trás".
    """
    def forward(self, input):
        """
        Observe que não há tipos. Não indicaremos expressamente os
        tipos de entradas que serão recebidas pelas camadas nem os
        tipos de saídas que elas retornarão.
        """
        raise NotImplemented

    def backward(self, gradient):
        """
        Da mesma forma, não indicaremos expressamente o formato do
        gradiente. Cabe ao usuário (você) avaliar se está fazendo as
        coisas de forma razoável.
        """
        raise NotImplemented

    def params(self):
        """
        Retorna os parâmetros desta camada. Como a implementação padrão
        não retorna nada, se houver uma camada sem parâmetros, você não
        precisará implementar isso.
        """
        return ()
    
    def grads(self) -> Iterable[Tensor]:
        """
        Retorna os gradientes na mesma ordem dos params().
        """
        return ()

In [75]:
import import_ipynb
from neural_networks import sigmoid

class Sigmoid(Layer):
    def forward(self, input: Tensor) -> Tensor:
        """
        Aplique a sigmoid em todos os elementos do tensor de 
        entrada e salve os resultados para usar na retropropagação.
        """
        self.sigmoids = tensor_apply(sigmoid, input)
        return self.sigmoids
    
    def backward(self, gradient: Tensor) -> Tensor:
        return tensor_combine(lambda sig, grad: sig * (1 - sig) * grad,
                              self.sigmoids,
                              gradient)

### A Camada Linear

In [60]:
import random

from probability import inverse_normal_cdf

def random_uniform(*dims: int) -> Tensor:
    if len(dims) == 1:
        return [random.random() for _ in range(dims[0])]
    else:
        return [random_uniform(*dims[1:]) for _ in range(dims[0])]

def random_normal(*dims: int,
                  mean: float = 0.0,
                  variance: float = 1.0) -> Tensor:
    if len(dims) == 1:
        return [mean + variance * inverse_normal_cdf(random.random())
                for _ in range(dims[0])]
    else:
        return [random_normal(*dims[1:], mean=mean, variance=variance)
                for _ in range(dims[0])]
    
assert shape(random_uniform(2, 3, 4)) == [2, 3, 4]
assert shape(random_normal(5, 6, mean=10)) == [5, 6]

In [15]:
def random_tensor(*dims: int, init: str = 'normal') -> Tensor:
    if init == 'normal':
        return random_normal(*dims)
    elif init == 'uniform':
        return random_uniform(*dims)
    elif init == 'xavier':
        variance = len(dims) / sum(dims)
        return random_normal(*dims, variance=variance)
    else:
        raise ValueError(f'unknown init: {init}')

In [88]:
from linear_algebra import dot

class Linear(Layer):
    def __init__(self, input_dim: int, output_dim: int, init: str = 'xavier') -> None:
        """
        A layer of output_dim neurons, each with input_dim weights
        (and a bias).
        """
        self.input_dim = input_dim
        self.output_dim = output_dim

        # self.w[o] is the weights for the o-th neuron
        self.w = random_tensor(output_dim, input_dim, init=init)

        # self.b[o] is the bias term for the o-th neuron
        self.b = random_tensor(output_dim, init=init)

    def forward(self, input: Tensor) -> Tensor:
        # Save the input to use in the backward pass.
        self.input = input

        # Return the vector of neuron outputs.
        return [dot(input, self.w[o]) + self.b[o]
                for o in range(self.output_dim)]

    def backward(self, gradient: Tensor) -> Tensor:
        # Each b[o] gets added to output[o], which means
        # the gradient of b is the same as the output gradient.
        self.b_grad = gradient

        # Each w[o][i] multiplies input[i] and gets added to output[o].
        # So its gradient is input[i] * gradient[o].
        self.w_grad = [[self.input[i] * gradient[o]
                        for i in range(self.input_dim)]
                       for o in range(self.output_dim)]

        # Each input[i] multiplies every w[o][i] and gets added to every
        # output[o]. So its gradient is the sum of w[o][i] * gradient[o]
        # across all the outputs.
        return [sum(self.w[o][i] * gradient[o] for o in range(self.output_dim))
                for i in range(self.input_dim)]

    def params(self) -> Iterable[Tensor]:
        return [self.w, self.b]

    def grads(self) -> Iterable[Tensor]:
        return [self.w_grad, self.b_grad]

### Redes Neurais Como Sequencias de Camadas

In [82]:
from typing import List

class Sequential(Layer):
    """
    Uma camada é uma sequência de outras camadas.
    Cabe a você avaliar se há coerência entre a saída de uma camada 
    e a entrada da próxima camada.
    """
    def __init__(self, layers: List[Layer]) -> None:
        self.layers = layers

    def forward(self, input):
        """Só avance a entrada pelas camadas em sequência."""
        for layer in self.layers:
            input = layer.forward(input)
        
        return input

    def backward(self, gradient):
        """Só retropropague o gradient pelas camadas na sequência inversa."""
        for layer in reversed(self.layers):
            gradient = layer.backward(gradient)
        return gradient

    def params(self) -> Iterable[Tensor]:
        """Só retorne os params de cada camada."""
        return (param for layer in self.layers for param in layer.params())

    def grads(self) -> Iterable[Tensor]:
        """Só retorne os grads de cada camada."""
        return (grad for layer in self.layers for grad in layer.grads())

In [63]:
# Represenando a rede neural que usamos para XOR

xor_net = Sequential([
    Linear(input_dim=2, output_dim=2),
    Sigmoid(),
    Linear(input_dim=2, output_dim=1),
    Sigmoid()
])

### Perda e Otimização

In [19]:
class Loss:
    def loss(self, predicted: Tensor, actual: Tensor) -> float:
        """Qual é a qualidade das previsões? (Os números maiores são piores)."""
        raise NotImplemented

    def gradient(self, predicted: Tensor, actual: Tensor) -> Tensor:
        """Como a perda muda a medida que mudam as previsões?"""
        raise NotImplemented

In [64]:
class SSE(Loss):
    """A função da perda que computa a soma dos erros quadráticos."""
    def loss(self, predicted: Tensor, actual: Tensor) -> float:
        # Compute o tensor das diferenças quadráticas
        squared_errors = tensor_combine(
            lambda predicted, actual: (predicted - actual) ** 2,
            predicted,
            actual
        )

        # E some tudo
        return tensor_sum(squared_errors)

    def gradient(self, predicted: Tensor, actual: Tensor) -> Tensor:
        return tensor_combine(
            lambda predicted, actual: 2 * (predicted - actual),
            predicted,
            actual
        )

In [65]:
class Optimizer:
    """
    O otimizador atualiza os pesos de uma camada (no local) usando informações
    conhecidas pela camada ou pelo otimizador (ou por ambos).
    """
    def step(self, layer: Layer) -> None:
        raise NotImplemented

In [66]:
class GradientDescent(Optimizer):
    def __init__(self, learning_rate: float = 0.1) -> None:
        self.lr = learning_rate

    def step(self, layer: Layer) -> None:
        for param, grad in zip(layer.params(), layer.grads()):
            # Atualize o param utilizando um passo de gradiente
            param[:] = tensor_combine(
                lambda param, grad: param - grad * self.lr,
                param,
                grad
            )
            

In [24]:
tensor = [[1, 2], [3, 4]]

for row in tensor:
    row = [0, 0]
assert tensor == [[1, 2], [3, 4]], 'a atribuição não atualiza a lista'

for row in tensor:
    row[:] = [0, 0]
assert tensor == [[0, 0], [0, 0]], 'mas a atribuição de fatia sim'

In [67]:
class Momentum(Optimizer):
    def __init__(self, learning_rate: float, momentum: float = 0.9) -> None:
        self.lr = learning_rate
        self.mo = momentum
        self.updates: List[Tensor] = []     # média móvel

    def step(self, layer: Layer) -> None:
        # Se não houver atualizações anteriores, começe com zeros
        if not self.updates:
            self.updates = [zero_likes(grad) for grad in layer.grads()]
        
        for update, param, grad in zip(self.updates, layer.params(), layer.grads()):
            # Aplique o momentum
            update[:] = tensor_combine(
                lambda u, g: self.mo * u + (1 - self.mo) * g,
                update,
                grad
            )

            # Em seguida, dê um passo de gradiente
            param[:] = tensor_combine(
                lambda p, u: p - self.lr * u,
                param,
                update
            )

In [89]:
xs = [[0., 0], [0., 1], [1., 0], [1., 1]]
ys = [[0.], [1.], [1.], [0.]]

random.seed(0)

net = Sequential([
    Linear(input_dim=2, output_dim=2),
    Sigmoid(),
    Linear(input_dim=2, output_dim=1)
])

In [90]:
import tqdm

optimizer = GradientDescent(learning_rate=0.1)
loss = SSE()

with tqdm.trange(3000) as t:
    for epoch in t:
        epoch_loss = 0.0

        for x, y in zip(xs, ys):
            predicted = net.forward(x)
            epoch_loss += loss.loss(predicted, y)
            gradient = loss.gradient(predicted, y)
            net.backward(gradient)

            optimizer.step(net)

        t.set_description(f"xor loss {epoch_loss:.3f}")

xor loss 0.000: 100%|██████████| 3000/3000 [00:10<00:00, 283.17it/s]


In [91]:
for param in net.params():
    print(param)

[[-1.6425160695224095, -1.4948117798303144], [-4.5676465720296635, -3.3649176350731893]]
[1.7673716823255186, 0.387270143794725]
[[3.198620479170404, -3.501803062142621]]
[-0.6462765963362219]
